In [16]:
import io
import nbformat
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import shuffle
import re
import csv

In [17]:
data = "train_data.csv"
df1 = pd.read_csv(data)

In [18]:
# classify majority and minority class
minority_class = df1[df1['label'] == 1]
majority_class = df1[df1['label'] == 0]

In [22]:
# calculate the difference between the two classes
nsamples = len(majority_class) - len(minority_class)

# get samples from the minority class
samples = minority_class.sample(n=nsamples, replace=True)

print(nsamples)
print(samples)

189076
            Id issuercountrycode   txvariantcode       bin    amount  \
101287  126643                MX        mccredit  528843.0   45400.0   
95993   120000                MX     visaclassic  493161.0  169915.0   
144037  180157                SE       visadebit  453903.0  110000.0   
87980   109908                MX     visaclassic  493172.0  279700.0   
112623  140759                GB       visadebit  476367.0   12700.0   
...        ...               ...             ...       ...       ...   
173270  216635                MX       visadebit  421316.0  129900.0   
7245      9086                GB       visadebit  475130.0    4000.0   
18004    22425                AU  visapurchasing  471514.0    8850.0   
11408    14226                AU     visaclassic  456471.0   90000.0   
22902    28558                MX    visaplatinum  410181.0  329900.0   

       currencycode shoppercountrycode shopperinteraction  \
101287          MXN                 MX          Ecommerce   
95993 

In [20]:
# find k nearest neighbors of each sample in the minority class
neigh = NearestNeighbors(n_neighbors=5)
nbrs = neigh.fit(minority_class)
distances, indices = nbrs.kneighbors(samples)

# create synthetic samples
synthetic_samples = []
for i in range(len(samples)):
    nn = indices[i]
    nn_index = np.random.choice(nn)
    diff = minority_class.iloc[nn_index] - samples.iloc[i]
    r = np.random.rand()
    synthetic_sample = samples.iloc[i] + r * diff
    synthetic_samples.append(synthetic_sample)
    
# convert the synthetic samples to a dataframe
synthetic_samples = pd.DataFrame(synthetic_samples, columns=df1.columns)

# add the synthetic samples to the minority class
oversampled = pd.concat([minority_class, synthetic_samples])

ValueError: could not convert string to float: 'AU'